<a href="https://colab.research.google.com/github/StarMindz/Cool-nerds-location-identifier/blob/main/ITU_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 6.8 MB/s 
     |████████████████████████████████| 182 kB 75.1 MB/s 
     |████████████████████████████████| 7.6 MB 75.9 MB/s 


In [ ]:
!pip install datasets
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 12.9 MB/s 
     |████████████████████████████████| 212 kB 62.6 MB/s 
     |████████████████████████████████| 132 kB 70.2 MB/s 
     |████████████████████████████████| 127 kB 73.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 1.7 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16179 sha256=1d1ea0bd7d73e6951e209de6082023eb1f87d98eea32ea90ac9aa65804fdfdba
  Stored in directory: /root/.cache/pip/wheels/ad/5c/ba/05fa33fa5855777b7d686e843ec07452f22a66a138e290e732
Successfully built seqeval


In [ ]:
!git clone https://github.com/rsuwaileh/IDRISI

Cloning into 'IDRISI'...
remote: Enumerating objects: 1062, done.
remote: Counting objects: 100% (188/188), done.
remote: Compressing objects: 100% (121/121), done.
remote: Total 1062 (delta 108), reused 100 (delta 59), pack-reused 874
Receiving objects: 100% (1062/1062), 26.15 MiB | 18.53 MiB/s, done.
Resolving deltas: 100% (325/325), done.
Checking out files: 100% (274/274), done.


In [ ]:
import numpy as np
import pandas as pd
import transformers
from datasets import load_metric
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
import torch
from datasets import Dataset
from sklearn.model_selection import train_test_split

In [ ]:

#Setup for calling data

import re
import glob
def get_data(typ, frmt):
  paths = []
  dataset_folder = '/content/IDRISI/LMR/data/EN/gold-'
  end_alias = f'{typ}-{frmt}'
  data_path = dataset_folder+end_alias
  for i,_ in enumerate(glob.glob(f'{data_path}/*')):
    paths.append(_)
  return paths

typ ='timebased'
# typ ='random'
# frmt = 'json'
frmt = 'bilou'

path_var = get_data(typ, frmt)

In [ ]:
# reads the BILOU formatted data

def read_bilou(path):
    tokens = []
    labels = []
    t = []
    l = []
    
    for token in open(path, encoding='utf-8').read().splitlines():
        if token == '':
            tokens.append(t)
            labels.append(l)
            t = []
            l = []
            continue
        splits = token.split()
        t.append(splits[0])
        l.append(splits[1])
        
    if len(t) > 0 and len(l) > 0:
        tokens.append(t)
        labels.append(l)
        
    return tokens, labels

def remove_punct(text):
    punct = re.compile(r'(\w+)')
    clean_text = ' '.join([m.group() for m in punct.finditer(text)])
    return clean_text

In [ ]:
#path_var

In [ ]:

#reading data from path
output = read_bilou(f'{path_var[0]}/train.txt')

In [ ]:
#output

In [ ]:

def getTrain (path_var):
  token = []
  tag = []
  for index in range(len(path_var)):
    output = read_bilou(f'{path_var[index]}/train.txt')
    token+=output[0]
    tag+=output[1]
  return (token, tag)

In [ ]:
train = getTrain(path_var)

In [ ]:
#train[1]

In [ ]:
def gettest (path_var):
  token = []
  tag = []
  for index in range(len(path_var)):
    output = read_bilou(f'{path_var[index]}/dev.txt')
    token+=(output[0])
    tag+=(output[1])
  return token, tag

In [ ]:
test = gettest(path_var)

In [ ]:
#test[0]

In [ ]:
##formating the data
trains = pd.DataFrame(columns=['tokens','tags'])
trains['tokens'],trains['tags'] = train

In [ ]:
##formating the data
tests = pd.DataFrame(columns=['tokens','tags'])
tests['tokens'],tests['tags'] = test

In [ ]:
trains.head()

,tokens,tags
0,"[RT, @UKinItaly, :, #, ItalyEarthquake, ,, how...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-O..."
1,"[RT, @cnni, :, JUST, IN, :, Death, toll, in, #...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,"[The, staff, of, MiraggioComics, went, to, don...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
3,"[Its, like, a, war, zone, a, woman, tells, me,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
4,"[Powerful, earthquake, in, #, Italy, kills, at...","[O, O, O, O, U-CTRY, O, O, O, O, O, O, O, O, O..."


In [ ]:
# for i in range(len(data['Word'])):
#   data['sentence'][i] = ' '.join(data['Word'][i])
#   data['word_labels'][i] = ','.join(data['Tags'][i])

In [ ]:
tests.head()

,tokens,tags
0,"[RT, @cctvnews, :, At, least, 250, people, dea...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1,"[RT, @UPI, :, #, ItalyEarthquake, update, :, A...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,"[RT, @AlisonBurnsJazz, :, Many, displaced, ani...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
3,"[Emergency, workers, ,, many, volunteers, ,, b...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
4,"[A, door, open, to, rubble, was, once, a, home...","[O, O, O, O, O, O, O, O, O, O, B-CITY, I-CITY,..."


In [ ]:
#x_train,x_test=train_test_split(data,test_size=0.20,shuffle=False)

In [ ]:
label_list_len = 13
label_list = ['CONT','CTRY','STAT','CNTY','CITY','DIST','ISL','NBHD','ST','NPOI','HPOI','OTHR','O']

In [ ]:
label_encoding = {'B-CITY':4,'B-CTRY':1,'B-DIST':5,'B-HPOI':10,'B-NBHD':7,'B-NPOI':9,'B-OTHR':11,'B-ST':8,'B-STAT':2,'I-CTRY':1,'I-HPOI':10,'I-NBHD':7,'I-NPOI':9,'I-OTHR':11,'I-ST':8,'L-CITY':4,'L-CTRY':1,'L-DIST':5,'L-HPOI':10,'L-NBHD':7,'L-NPOI':9,'L-OTHR':11,'L-ST':8,'L-STAT':2,'O':12,'U-CITY':4,'U-CNTY':3,'U-CTRY':1,'U-DIST':5,'U-HPOI':10,'U-ISL':6,'U-NBHD':7,'I-DIST':5,'U-ST':8,'L-CONT':0,'B-CONT':0,'I-STAT':2,'I-CNTY':3,'I-ISL':6,'L-ISL':6,'B-ISL':6,'L-CNTY':3,'U-NPOI':9,'B-CNTY':3,'U-OTHR':11,'U-CONT':0,'I-CITY':4,'U-STAT':2}
len(label_encoding)

48

In [ ]:
problem = "ner" 
model_name = "distilbert-base-uncased"
batch_size = 8

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
trains = Dataset.from_pandas(trains)
tests = Dataset.from_pandas(tests)

In [ ]:
trains

Dataset({
    features: ['tokens', 'tags'],
    num_rows: 14392
})

In [ ]:
def tokenize_and_align_labels(examples):
    label_all_tokens = True
    tokenized_inputs = tokenizer(list(examples["tokens"]), truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples["tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif label[word_idx] == '0':
                label_ids.append(0)
            elif word_idx != previous_word_idx:
                label_ids.append(label_encoding[label[word_idx]])
            else:
                label_ids.append(label_encoding[label[word_idx]] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)
        
    tokenized_inputs["labels"] = labels
    return tokenized_inputs


train_tokenized = trains.map(tokenize_and_align_labels, batched=True)
test_tokenized = tests.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/15 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [ ]:
train_tokenized[1]

{'tokens': ['RT',
  '@cnni',
  ':',
  'JUST',
  'IN',
  ':',
  'Death',
  'toll',
  'in',
  '#',
  'ItalyEarthquake',
  'is',
  'now',
  'at',
  '73',
  ',',
  'according',
  'to',
  'the',
  'Civil',
  'Protection',
  'Agency',
  '#',
  'terremoto'],
 'tags': ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 'input_ids': [101,
  19387,
  1030,
  13229,
  2072,
  1024,
  2074,
  1999,
  1024,
  2331,
  9565,
  1999,
  1001,
  3304,
  14644,
  2705,
  16211,
  3489,
  2003,
  2085,
  2012,
  6421,
  1010,
  2429,
  2000,
  1996,
  2942,
  3860,
  4034,
  1001,
  25170,
  15319,
  102],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'labels': [-100,
  12,
  12,
  12,
  12,
  12,
  12,
  12,
  12,
  12,
  12,
  12,
  12,
  12,
  12,
  1

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=label_list_len)

args = TrainingArguments(
    f"test-{problem}",
    evaluation_strategy = "epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=1e-5,
)

data_collator = DataCollatorForTokenClassification(tokenizer)
metric = load_metric("seqeval")


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [[label_list[p] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    true_labels = [[label_list[l] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {"precision": results["overall_precision"], "recall": results["overall_recall"], "f1": results["overall_f1"], "accuracy": results["overall_accuracy"]}
    
trainer = Trainer(
    model,
    args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()
trainer.evaluate()
trainer.save_model('un-ner.model')

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN t

The following columns in the training set don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: tokens, tags. If tokens, tags are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 14392
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 5397
  Number of trainable parameters = 66372877
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.073400,0.073628,0.741292,0.824195,0.780548,0.979914
2,0.053200,0.066683,0.787879,0.825936,0.806458,0.982272
3,0.032200,0.074716,0.787510,0.817668,0.802306,0.981397


Saving model checkpoint to test-ner/checkpoint-500
Configuration saved in test-ner/checkpoint-500/config.json
Model weights saved in test-ner/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-ner/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-ner/checkpoint-500/special_tokens_map.json
Saving model checkpoint to test-ner/checkpoint-1000
Configuration saved in test-ner/checkpoint-1000/config.json
Model weights saved in test-ner/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in test-ner/checkpoint-1000/tokenizer_config.json
Special tokens file saved in test-ner/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to test-ner/checkpoint-1500
Configuration saved in test-ner/checkpoint-1500/config.json
Model weights saved in test-ner/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in test-ner/checkpoint-1500/tokenizer_config.json
Special tokens file saved in test-ner/checkpoint-1500/special_tokens_map.json
The

Saving model checkpoint to un-ner.model
Configuration saved in un-ner.model/config.json
Model weights saved in un-ner.model/pytorch_model.bin
tokenizer config file saved in un-ner.model/tokenizer_config.json
Special tokens file saved in un-ner.model/special_tokens_map.json


In [ ]:
def predict_loc(text):

    """
        Predicts location in texts and
        returns their mentioned location
    """

    tokenizer = AutoTokenizer.from_pretrained('./un-ner.model/')
    
    tokens = tokenizer(text)
    torch.tensor(tokens['input_ids']).unsqueeze(0).size()
    
    model = AutoModelForTokenClassification.from_pretrained('./un-ner.model/', num_labels=len(label_list))
    predictions = model.forward(input_ids=torch.tensor(tokens['input_ids']).unsqueeze(0), attention_mask=torch.tensor(tokens['attention_mask']).unsqueeze(0))
    predictions = torch.argmax(predictions.logits.squeeze(), axis=1)
    predictions = [label_list[i] for i in predictions]
    
    words = tokenizer.batch_decode(tokens['input_ids'])
    out_labeled = {x:y for x,y in zip(words, predictions)}

    return out_labeled

In [ ]:
## formating output to json format
import json

# test input_path = "/content/IDRISI/LMR/data/EN/gold-random-json/california_wildfires_2018/dev.jsonl"


def load_input_tweets(input_path):
    """function that reads the inputed tweets"""
    tweets = {}
    for line in open(input_path, encoding='utf-8').read().splitlines(): 
        tweet = json.loads(line)
        tweets[tweet["tweet_id"]] = tweet["text"]
    return tweets


In [ ]:
import re

def model_output(tweets, path):
    """
       writes the model output to a json file
    """
    with open('output.json', 'w') as file:
        
        for i in range(len(tweets)):
            output = {}
            output["tweet_id"] = list(tweets.keys())[i]
            text = tweets[list(tweets.keys())[i]]
            prediction = predict_loc(text)
            locations = []
            for i in range(len(prediction)):
                if list(prediction.values())[i] != 'O':
                    word = list(prediction.keys())[i]
                    if word not in text.lower():
                        continue
                    start, end = re.search(word, text.lower()).span()
                    locations.append({"text":text[start:end], "start_offset": start, "end_offset": end})
            output["location_mentions"] = locations
            file.write(json.dumps(output) + "\n")



In [ ]:
## creating input dataset for  format checker with our jsonl dev dataset

input_path = "/content/IDRISI/LMR/data/EN/gold-random-json/california_wildfires_2018/dev.jsonl"


def load_input_tweet(input_path):
    """function that reads the inputed tweets"""
    tweets = {}

    for line in open(input_path, encoding='utf-8').read().splitlines(): 
        tweet = json.loads(line)
        tweets[tweet["tweet_id"]] = tweet["text"]
    with open ('input.json','w') as file:
      
        for i in range(len(tweets)):
            output = {}
            output["tweet_id"] = list(tweets.keys())[i]
            output["text"] = list(tweets.values())[i]

            file.write(json.dumps(output)+'\n')


load_input_tweets(input_path)
        



In [ ]:
def predict(input_path, output_path):
    """
        make the lmr prediction
    """
    tweets = load_input_tweets(input_path)

    model_output(tweets, output_path)

    print("Model finished predictions!!!")

ptedict('/content/input.json', "/content/output.json")

Streaming output truncated to the last 5000 lines.
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_10": 10,
    "LABEL_11": 11,
    "LABEL_12": 12,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_7": 7,
    "LABEL_8": 8,
    "LABEL_9": 9
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.25.1",
  "vocab_size": 30522
}

loading weights file ./un-ner.model/pytorch_model.bin
All 

Model finished predictions!!!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!python3 /content/IDRISI/LMR/format_checker/format_checker.py --input_path /content/input.json --output_path /content/output.json

The output file had passed all checks successfully!
